In [4]:
import pandas as pd
import json

import os
from config import DATA_PATH

author_prefix_lst = ['#index', '#n', '#a', '#pc', '#cn', '#hi', '#pi', '#upi', '#t']
author_cols = ['author_id', 'author_name', 'affiliations', 'published count', 'citation num',
               'H-index', 'P-index of equal A', 'P-index of unequal A', 'interests']

paper_prefix_lst = ['#index', '#*', '#@', '#t', '#c', '#%']
paper_cols = ['id', 'title', 'authors_name', 'year', 'venue_name', 'references']

JSON_KEYWORDS = ['id', 'authors', 'venue', 'year', 'keywords',
                 'references', 'n_citation', 'doc_type']
JSON_COLUMNS = ['id', 'author_id', 'author_org', 'venue_id', ' venue_name',
                'year', 'keywords', 'references', 'n_cites', 'doc_type']

In [2]:
def parsejson2df(file, col_lst=JSON_KEYWORDS):
    """ Parse & load json file to Dataframe.

    Data at https://originalstatic.aminer.cn/misc/dblp.v12.7z
    ** THIS FILE IS PRETTY LARGE (12GB when unzipped), the file is loaded by readline,
    HOWEVER THE JSON DATA IS CONVERTED TO A SINGLE DATAFRAME, MAY REQUIRE LARGE MEMORY.

    ETA 5hrs
    """

    def parse(js):
        processed_line = dict()
        for col in set(col_lst).intersection(set(js.keys())):
            # Only keywords in the list above are considered
            # If more keywords are required, according read_in code should be added
            if col == 'id':
                processed_line['id'] = js['id']
            elif col == 'authors':
                authors = js['authors']
                processed_line['authors_id'] = ';'.join([str(author.get('id', None))
                                                         for author in authors if author.get('id', None)])
                processed_line['authors_name'] = ';'.join([author.get('name', None)
                                                           for author in authors if author.get('name', None)])
                processed_line['authors_org'] = ';'.join([str(author.get('org', None))
                                                          for author in authors if author.get('org', None)])
            elif col == 'venue':
                venue = js['venue']
                processed_line['venue_id'] = venue.get('id', None)
                processed_line['venue_name'] = venue.get('raw', None)
            elif col == 'year':
                processed_line['year'] = js['year']
            elif col == 'keywords':
                processed_line['keywords'] = ';'.join(js['keywords'])
            elif col == 'references':
                processed_line['references'] = ';'.join([str(r) for r in js['references']])
            elif col == 'n_citation':
                processed_line['n_cites'] = js['n_citation']
            elif col == 'doc_type':
                processed_line['doc_type'] = js['doc_type']
        return processed_line

    parsed_df = pd.DataFrame()
    with open(file, encoding='utf-8') as f:
        i = 0
        while i < 10000:
            line = f.readline()
            if not line:
                break
            line = line.lstrip('[')
            line = line.lstrip(',')
            line = line.rstrip(']')
            line = line.rstrip('\n')
            if line:
                js = json.loads(line)
                parsed_df = parsed_df.append(parse(js), ignore_index=True)
            i += 1
    # Change data type
    parsed_df['id'] = parsed_df['id'].astype('int64')
    parsed_df['year'] = parsed_df['year'].astype('int')
    parsed_df['n_cites'] = parsed_df['n_cites'].astype('int')

    return parsed_df

In [3]:
import time
start = time.time()
dblp_paper_df = parsejson2df(os.path.join(DATA_PATH, 'dblp.v12.json'), col_lst=JSON_KEYWORDS)
print(time.time() - start)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Dataset\\ThinkTank\\dblp.v12.json'

In [ ]:
import os

In [ ]:
dblp_paper_df.to_csv(os.path.join(DATA_PATH, 'dblp_paper.csv'), index=False)

In [ ]:
with open((os.path.join(DATA_PATH, 'dblp.v12.json')), encoding='utf-8') as f:
    i = 0
    while i < 5:
        line = f.readline()
        i += 1

In [ ]:
js = json.loads(line)

In [ ]:
from sklearn.decomposition import PCA
